# Auto Scraper


In [1]:
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import re
import statistics
import undetected_chromedriver as uc

In [2]:
def initiate (enlace=0):
    #driver = webdriver.Chrome(ChromeDriverManager().install())
    driver = uc.Chrome()
    driver.implicitly_wait(5)
    
    driver.get('https://www.fotocasa.es/es/')
    time.sleep(3)
    driver.maximize_window()
    time.sleep(2)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Aceptar y cerrar')]"))).click()
    time.sleep(3)
    return driver
    

In [3]:
#driver= initiate()

In [4]:
def buscar_distrito(distrito, driver):
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.CLASS_NAME,'sui-AtomInput-input sui-AtomInput-input-size-m'.replace(' ', '.')))).click()
    time.sleep(3)
    
    buscar= driver.find_element(by=By.CLASS_NAME,value="sui-AtomInput-input sui-AtomInput-input-size-m".replace(' ', '.'))
    buscar.send_keys(distrito)
    time.sleep(6)
    WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Buscar')]"))).click()

In [5]:
#buscar_distrito('san esteban leon',driver)

In [6]:
def scroll_down(driver):  
    elem = driver.find_element(By.TAG_NAME,value="body")
    no_of_pagedowns = 25
    time.sleep(3)
    while no_of_pagedowns:
        elem.send_keys(Keys.PAGE_DOWN)
        time.sleep(0.5)
        no_of_pagedowns-=1
        
    time.sleep(3)
    element = driver.find_element(By.CLASS_NAME,'sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded'.replace(' ', '.'))
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()   

In [7]:
#scroll_down(driver)

In [8]:
def get_html(driver):
    html_txt=driver.page_source
    soup=BeautifulSoup(html_txt)
    viviendas= soup.find('section',class_='re-SearchResult')
    return viviendas

In [9]:
#viviendas=get_html(driver)

In [10]:
def get_data(viviendas,distrito):
    try:
        vivienda_str_list=[]

        for vivienda in viviendas:     
            vivienda_str_len=len(str(vivienda))
            vivienda_str_list.append(vivienda_str_len)

        mean_len=statistics.mean(vivienda_str_list)*0.80
        print('Mean length value of the html:',mean_len)
        
        precios,titulos,habitaciones,baños,superficie,ascensor,planta,link,fechas = get_vivienda_data(viviendas, mean_len)

        df=pd.DataFrame(list(zip(titulos,precios,habitaciones,planta,superficie,baños,ascensor,fechas ,link))).drop_duplicates()
        df=df.rename(columns={0:"Título",1:"Precio",2:"Habitaciones",3:"Planta",4:"Superficie",5:"Baños",6:"Ascensor",7:"Fecha Publicación",8:"Links"})
        ciudades = ['Leon', 'Alicante']
        df['Barrio']=distrito
        for ciudad in ciudades:
            if ciudad in distrito: 
                df['Ciudad']=ciudad
    except:
        df=pd.DataFrame()
    return df
    
    
def get_vivienda_data(viviendas, mean_len):   
    lista_precios=[]
    lista_titulos=[]
    lista_habitaciones=[]
    lista_baños=[]
    lista_superficie=[]
    lista_ascensor = []
    lista_planta = []
    lista_links = []
    lista_fechas = []
    
    for vivienda in viviendas:
        try:
            #data required
            vivienda_str = str(vivienda)
            hab_pos = re.search(r'habs.', vivienda_str)
            hab_pos_indi = re.search(r'hab.', vivienda_str)
    
            baño_pos = re.search(r'(baño|Baño)', vivienda_str)
    
            superficie_pos = re.search(r'([0-9] [m])', vivienda_str)
            keyword='ascensor'

            planta='Planta'
            planta_baja='planta'
            
            #precios
            precio = vivienda.find('span', class_='re-CardPrice').getText()
            lista_precios.append(precio)
            
            #titulos
            titulos = vivienda.find('span', class_='re-CardTitle').getText()
            lista_titulos.append(titulos)
            
            #habitaciones
            if hab_pos:
                hab_pos_text = hab_pos.start()
                hab = vivienda_str[hab_pos_text-2:hab_pos_text-1]
                lista_habitaciones.append(hab)
            elif hab_pos_indi:
                hab_pos_text = hab_pos_indi.start()
                hab = vivienda_str[hab_pos_text-2:hab_pos_text-1]
                lista_habitaciones.append(hab)       
            else:
                lista_habitaciones.append(None)
            
            
            #baños
            if baño_pos:
                baño_pos_text = baño_pos.start()
                b = vivienda_str[baño_pos_text-2:baño_pos_text-1]
                lista_baños.append(b)
            else:
                lista_baños.append(None)
            
            
            #superficie
            if superficie_pos:
                superficie_pos_text = superficie_pos.start()
                s = vivienda_str[superficie_pos_text-2:superficie_pos_text+3]

                if 'm' not in s:
                    lista_superficie.append(None)
                else:
                    lista_superficie.append(re.findall(r'\d+', s)[0])
            else:
                lista_superficie.append(None) 
            
            #ascensor
            if keyword in vivienda_str:
                lista_ascensor.append('SI')

            else:
                lista_ascensor.append('NO') 
                
            #planta
            
            if planta in vivienda_str:
                planta_pos=re.search(r'(Planta)',vivienda_str)
                planta_pos_text=planta_pos.start()
                p=vivienda_str[planta_pos_text-3:planta_pos_text-2]
                lista_planta.append(p)        

            elif planta_baja in vivienda_str:
                planta_pos_baja=re.search(r'(planta)',vivienda_str)
                planta_pos_text_baja=planta_pos_baja.start()
                lista_planta.append(0)        

            else:
                lista_planta.append(None)
            
            #link
            links = vivienda.find('a',href=True)
            href = links['href']
            fotocasa = 'https://www.fotocasa.es/'
            if '/es/comprar/vivienda/' in href:
                lista_links.append(fotocasa+href)
            
            
            #fecha
            
            fecha=vivienda.find('span',class_='re-CardTimeAgo').getText()
            if re.search(r'\d+', fecha):
                fecha_int=int(re.search(r'\d+', fecha).group())
            else:
                fecha_int=0
            lista_fechas.append(fecha_int)
            
                
        except AttributeError:
            pass
    
    return lista_precios, lista_titulos,lista_habitaciones,lista_baños,lista_superficie,lista_ascensor,lista_planta,lista_links,lista_fechas

In [11]:
#df=get_data(viviendas)


In [12]:
def get_max_value(driver):
    resultSet = driver.find_element(By.XPATH,"//ul[@class='sui-MoleculePagination']")
    list_pages=[]
    all_li = resultSet.find_elements(By.TAG_NAME,"li")
    for li in all_li:
        text = li.text
        try:
            text=int(text)
            list_pages.append(text)
        except:
            pass
        
    max_value=max(list_pages)
    return max_value

In [13]:
#max_value=get_max_value(driver)

In [14]:
def next_page(driver):
    try:
        time.sleep(3)
        buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
        time.sleep(7)
    except:
        time.sleep(3)
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, "//button[@class='ab-close-button']"))).click()
        time.sleep(3)
        buttons = driver.find_element(By.XPATH,"//ul/li[@class='sui-MoleculePagination-item'][last()]/a[@class='sui-AtomButton sui-AtomButton--primary sui-AtomButton--outline sui-AtomButton--center sui-AtomButton--small sui-AtomButton--link sui-AtomButton--empty sui-AtomButton--rounded']")
        WebDriverWait(driver, 5).until(EC.element_to_be_clickable(buttons)).click()
        time.sleep(7)

In [15]:
#next_page(driver)

In [16]:
#scroll_down(driver)

In [17]:
#viviendas=get_html(driver)

In [18]:
#n_df=get_data(viviendas)

In [19]:
#df = pd.concat([df,n_df],axis=0)

In [20]:
#df.shape

In [21]:
def batch(distrito):
    driver = initiate()
    buscar_distrito(distrito,driver)
    scroll_down(driver)
    max_value=get_max_value(driver)
    viviendas = get_html(driver)
    df=get_data(viviendas,distrito)
    for i in range(max_value-1):
        next_page(driver)
        scroll_down(driver)
        viviendas=get_html(driver)
        n_df=get_data(viviendas,distrito)
        print(n_df.shape)
        df = pd.concat([df,n_df],axis=0)
        print(df.shape)
        
    return df


In [23]:
import concurrent.futures


    

In [29]:
if __name__ == '__main__':
    busquedas = ['Carolinas Bajas Alicante','San Mames - La Palomera Leon','Chantría la lastra Leon','San Esteban Leon Capital','Pla del Bon Repos - La Goteta Alicante']
    df = pd.DataFrame()

    with concurrent.futures.ThreadPoolExecutor(max_workers=2) as executor:
        future_data = [executor.submit(batch, busqueda) for busqueda in busquedas]

        while future_data:
            done, future_data = concurrent.futures.wait(future_data, return_when=concurrent.futures.FIRST_COMPLETED)
            for future in done:
                data = future.result()
                df = pd.concat([df, data], axis=0)
                print(df.shape)

Mean length value of the html: 6301.129411764706
Mean length value of the html: 6978.375757575757
Mean length value of the html: 5718.8848484848495
(30, 11)
(61, 11)
Mean length value of the html: 6399.200000000001
(30, 11)
(60, 11)
Mean length value of the html: 6099.806060606061
(30, 11)
(91, 11)
Mean length value of the html: 5386.400000000001
(30, 11)
(90, 11)
Mean length value of the html: 5537.381818181819
(30, 11)
(121, 11)
Mean length value of the html: 5243.587878787879
(30, 11)
(120, 11)
Mean length value of the html: 3703.2000000000003
(2, 11)
(123, 11)
(123, 11)
Mean length value of the html: 5531.587878787879
(30, 11)
(150, 11)
Mean length value of the html: 5636.848484848485
(30, 11)
(180, 11)
Mean length value of the html: 6369.435294117648
Mean length value of the html: 6053.090909090909
(30, 11)
(210, 11)
Mean length value of the html: 5638.2787878787885
(30, 11)
(61, 11)
Mean length value of the html: 5719.466666666667
(30, 11)
(240, 11)
Mean length value of the html:

In [30]:
df

,Título,Precio,Habitaciones,Planta,Superficie,Baños,Ascensor,Fecha Publicación,Links,Barrio,Ciudad
0,Piso con ascensor en Carolinas Bajas,139.000 €,2,None,90,1,SI,225,https://www.fotocasa.es//es/comprar/vivienda/a...,Carolinas Bajas Alicante,Alicante
1,Piso con balcón en Carolinas Bajas,115.000 €,2,3,74,1,NO,8,https://www.fotocasa.es//es/comprar/vivienda/a...,Carolinas Bajas Alicante,Alicante
2,"Piso con terraza en Carrer de Torres Quevedo, ...",115.000 €,3,0,76,1,SI,0,https://www.fotocasa.es//es/comprar/vivienda/a...,Carolinas Bajas Alicante,Alicante
3,Piso con balcón en Carolinas Bajas,94.000 €,3,1,85,1,SI,3,https://www.fotocasa.es//es/comprar/vivienda/a...,Carolinas Bajas Alicante,Alicante
4,Piso con aire acondicionado en Carolinas Bajas,146.500 €,4,2,110,2,NO,31,https://www.fotocasa.es//es/comprar/vivienda/a...,Carolinas Bajas Alicante,Alicante
...,...,...,...,...,...,...,...,...,...,...,...
5,"Piso en Calle Maestro Luis Torregrosa, Pla de ...",115.000 €,3,None,82,1,SI,738,https://www.fotocasa.es//es/comprar/vivienda/a...,Pla del Bon Repos - La Goteta Alicante,Alicante
6,Casa o chalet con 2 baños en Pla de Bon Repós,205.000 €,4,0,None,2,NO,134,https://www.fotocasa.es//es/comprar/vivienda/a...,Pla del Bon Repos - La Goteta Alicante,Alicante
7,Piso con balcón en Pla de Bon Repós,116.000 €,3,5,110,1,SI,4,https://www.fotocasa.es//es/comprar/vivienda/a...,Pla del Bon Repos - La Goteta Alicante,Alicante
8,"Piso en Calle Padre Esplá, Pla de Bon Repós",112.500 €,2,None,50,1,SI,646,https://www.fotocasa.es//es/comprar/vivienda/a...,Pla del Bon Repos - La Goteta Alicante,Alicante


In [31]:
df.to_csv('for_sale.csv')